In [2]:
from pathlib import Path

from pandas import Timedelta, Timestamp, read_csv, concat

from osekit.utils.timestamp_utils import strptime_from_text

from post_processing.utils.glider_utils import load_glider_nav, plot_detections_with_nav_data, \
    compute_acoustic_diversity, export_gpx
import matplotlib.pyplot as plt

### Load data

In [ ]:
input_dir = Path(r"L:\acoustock\Bioacoustique\DATASETS\GLIDER\GLIDER SEA034\MISSION_58_OHAGEODAMS_2024\APRES_MISSION\NAV")
nav = load_glider_nav(input_dir)
nav = nav[nav['Timestamp'] > Timestamp('2024-02-21T00:00:00+0000')]

df_BF = read_csv(Path(r"L:\acoustock\Bioacoustique\DATASETS\GLIDER\GLIDER SEA034\MISSION_58_OHAGEODAMS_2024\ANALYSES\APLOSE\results\GLIDER_OHAGEODAMS_BF_results.csv"), parse_dates=['start_datetime', 'end_datetime']).dropna(subset=['annotation']).reset_index(drop=True)
df_HF = read_csv(Path(r"L:\acoustock\Bioacoustique\DATASETS\GLIDER\GLIDER SEA034\MISSION_58_OHAGEODAMS_2024\ANALYSES\APLOSE\results\GLIDER_OHAGEODAMS_HF_results.csv"), parse_dates=['start_datetime', 'end_datetime']).dropna(subset=['annotation']).reset_index(drop=True)
df = concat([df_BF, df_HF]).sort_values('start_datetime')

### Plot detection with nav data

In [ ]:
ticks = Timedelta('1d')
plot_detections_with_nav_data(df=df_HF[df_HF['annotation'].isin(["Sperm_Whale", "Delphinids clics"])], nav=nav, criterion='Depth', ticks=ticks)
plt.show()

plot_detections_with_nav_data(df=df_HF, nav=nav, criterion='Depth', ticks=ticks)
plt.show()

### Compute acoustic diversity

In [ ]:
task_status_filenames = read_csv(r"L:\acoustock\Bioacoustique\DATASETS\GLIDER\GLIDER SEA034\MISSION_58_OHAGEODAMS_2024\ANALYSES\APLOSE\BF\GLIDER_OHAGEODAMS_BF_task_status.csv")['filename']
time_vector = [strptime_from_text(f, "%Y_%m_%d_%H_%M_%S").tz_localize('UTC') for f in task_status_filenames]

df_acoustic_div = compute_acoustic_diversity(df=df, nav=nav, time_vector=time_vector)

# df_acoustic_div.to_csv(Path(r'\output\path\acoustic_diversity.csv'), index=False)

### Export navigation data to GPX

In [ ]:
export_gpx(nav=nav, output_dir=Path(r"\output\path"), output_file="test_trace")